In [ ]:
# Ensamblaje de modelos = Ensemble Learning es un método de Machine Learning en el que múltiples modelos - normalmente llamados weak learners (modelos débiles) son entrenados para resolver el mismo problema y
# combinados para obtener una predicción conjunta que produzca mejores resultados que las predicciones por separado

In [ ]:
#Bagging = bootstraping de datos y aggregation de las predicciones

In [ ]:
#Boosting = se entrena el modelo secuencialmente, tienen más tendencia a sobreajustar (tienden al poco sesgo), mientras que el bagging tiende a reducir la varianza

In [ ]:
#Boosting: Se empieza por la media, mide el error de cada punto, entrena un nuevo modelo entonces entrenando los errores

In [ ]:
#sesgo = poco sesgo es que me equivoco poco (cerca de la diana)

In [ ]:
#XGBoost = Extreme Gradient Boosting, la diferencia con un gradient boosting es que para escoger sus particiones (scores) lo hace de una manera específica
#y que se pueden escoger una serie de hiperparametros  que te ayudan a reducir el sobreajuste

# 1. INTRODUCCIÓN: Gradient Boosting
Introducción a modelos de agregación de árboles de decisión con Gradient Boosting.




## 1.1 Qué es un Gradient Boosting?
Un tipo de algoritmo de aprendizaje supervisado que se basa la agregación en serie de árboles de decisión.

## 1.2 Import
Importamos todas las librerías necesarias para este análisis: pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from sklearn import tree
from sklearn import ensemble

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 1.3 Carga el dataset del Titanic
Volvamos con el Dataset del Titanic

Vamos a evaluar el rendimiento de un Random Forest en el dataset del Titanic (utilizando anteriormente para evaluar árboles de decisión)

Este es el link al archivo raw: https://raw.githubusercontent.com/JimenaAreta/thevalley-MDS/jimena/datasets/titanic.csv.

Importa los datos en un dataframe llamado **dataframe df_titanic**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JimenaAreta/thevalley-MDS/jimena/datasets/titanic.csv'

# Importa csv
df_titanic = pd.read_csv(url)

# Visualización primeras filas
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1,4 Limpieza del Dataset
Limpiamos los nulos y creamos la variable Title

In [ ]:
# Creamos una nueva variable Título (es opcional, pero podría ayudar al modelo)
df_titanic['Title'] = [name.split(",")[1].split(".")[0][1:] for name in df_titanic['Name']]
df_titanic.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs


In [ ]:
# Columnas con las que nos quedamos
cols = ['Survived',
        'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']

# De ellas, cuáles son categóricas
cat_cols = ['Sex', 'Embarked', 'Title']

# Visualizamos las columnas con las que nos hemos quedado
df_titanic[cols].head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


In [ ]:
# Rellenamos los nulos de las variables numéricas (edad)
df_titanic[cols].isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
Title         0
dtype: int64

In [ ]:
# Rellenamos los nulos de la variable edad de una forma avanzada:
# calculando la media para cada título

filtro_edad_nula = df_titanic['Age'].isna()
filtro_edad_no_nula = df_titanic['Age'].notnull()

# Rellenamos los nulos en función del título
for titulo in df_titanic['Title'].unique():
  filtro_titulo = df_titanic['Title'] == titulo

  if len(df_titanic[((filtro_titulo) & (filtro_edad_no_nula))]) > 2:
    print(titulo)
    df_titanic.loc[((filtro_titulo)&(filtro_edad_nula)), 'Age'] = df_titanic.loc[((filtro_titulo)&(filtro_edad_no_nula)), 'Age'].median()

# Rellenamos los que sigan siendo nulos (títulos solo presentes entre gente sin la edad informada)
df_titanic['Age'] = df_titanic['Age'].fillna(df_titanic['Age'].median())

Mr
Mrs
Miss
Master
Rev
Dr


In [ ]:
# Eliminamos los nulos de la variable Embarked
print(len(df_titanic))
df_titanic = df_titanic.dropna(subset=['Embarked'])
print(len(df_titanic))

891
889


In [ ]:
# Dumificamos variables categóricas
df_titanic_i = pd.get_dummies(df_titanic[cols],
                              prefix_sep='_',
                              drop_first=True,
                              columns=cat_cols)
df_titanic_i.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Title_Col,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,0,3,22.0,1,0,7.2500,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,38.0,1,0,71.2833,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,35.0,1,0,53.1000,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,0,3,35.0,0,0,8.0500,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0


## 1.5 Train-Test Split

In [ ]:
# Generamos las matrices X e y
X_titanic = df_titanic_i.drop('Survived',axis=1)
y_titanic = df_titanic_i['Survived']

X_titanic.shape, y_titanic.shape

((889, 24), (889,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic,
                                                    y_titanic,
                                                    test_size=0.25,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((666, 24), (223, 24), (666,), (223,))

# 2. Entrenamiento árbol de decisión
Entrenamos un árbol de decisión tomando medidas para evitar el overfitting.

In [ ]:
def entrenar_modelo_y_predecir_classificacion(modelo):
  # Entreno el árbol con el set de entrenamiento
  modelo = modelo.fit(X=X_train, y=y_train)
  # Uso el árbol para predecir sobre el dataset de entrenamiento
  y_pred_train = modelo.predict(X_train)
  # Uso el árbol para predecir sobre el dataset de test
  y_pred_test = modelo.predict(X_test)
  # Cómo de buena es la predicción?
  ac_train = round(accuracy_score(y_train, y_pred_train), 4)
  print('Precisión en set de entrenamiento :', ac_train)
  ac_test = round(accuracy_score(y_test, y_pred_test), 4)
  print('Precisión en set de test :', ac_test)
  print('Degradación: ', round((ac_train-ac_test)/ac_train*100,2), '%')

In [ ]:
arbol_decision = tree.DecisionTreeClassifier(max_depth=20,
                                             min_samples_split = 20,
                                             min_samples_leaf = 5,
                                             min_impurity_decrease = 0.003)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(arbol_decision)

Precisión en set de entrenamiento : 0.8589
Precisión en set de test : 0.843
Degradación:  1.85 %


# 3. Entrenamiento Random Forest
Ahora comparamos el rendimiento del árbol de decisión con un Random Forest.
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
random_forest = ensemble.RandomForestClassifier(n_estimators = 200,
                                                max_depth = 3,
                                                min_samples_split = 10,
                                                min_samples_leaf = 5)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(random_forest)

Precisión en set de entrenamiento : 0.8318
Precisión en set de test : 0.7937
Degradación:  4.58 %


# 4. Gradient Boosting
Utilizaremos de nuevo la implementación de sklearn, que podéis ver aquí: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html.

In [ ]:
grad_boost = ensemble.GradientBoostingClassifier()

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(grad_boost)

print()
grad_boost

Precisión en set de entrenamiento : 0.8994
Precisión en set de test : 0.8206
Degradación:  8.76 %



GradientBoostingClassifier()

In [ ]:
grad_boost = ensemble.GradientBoostingClassifier(n_estimators = 100,
                                                 learning_rate = 0.1,
                                                 max_depth = 5,
                                                 min_samples_split = 20,
                                                 min_samples_leaf = 5,
                                                 min_impurity_decrease = 0.01,
                                                 random_state = 0)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(grad_boost)

Precisión en set de entrenamiento : 0.9354
Precisión en set de test : 0.8341
Degradación:  10.83 %


# 5. EJERCICIO

## 5.1 Importa Dataset Diabetes
Vamos a utilizar el dataset de Diabetes.

Impórtalo utilizando el archivo raw: 'https://raw.githubusercontent.com/JimenaAreta/thevalley-MDS/jimena/datasets/diabetesIndia.csv'.

Guárdalo en el **dataframe df**.

In [ ]:
url = 'https://raw.githubusercontent.com/JimenaAreta/thevalley-MDS/jimena/datasets/diabetesIndia.csv'

df = pd.read_csv(url)

df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


## 5.2 Limpia el dataset
Comprueba el dataset (hay nulos) y prepáralo si es necesario.

In [ ]:
# Rellenamos los nulos de las variables numéricas (edad)
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

## 5.3 Train-Test split
Genera el set de pruebas (test).

In [ ]:
# Generamos las matrices X e y
X = df.drop('Outcome',axis=1)
y = df['Outcome']

X.shape, y.shape

((768, 8), (768,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((576, 8), (192, 8), (576,), (192,))

## 5.4 Árbol de Decisión
Encuentra el mejor árbol de decisión posible

In [ ]:
modelo = tree.DecisionTreeClassifier(max_depth=15,
                                     min_samples_split = 20,
                                     min_samples_leaf = 10,
                                     min_impurity_decrease = 0.005)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

Precisión en set de entrenamiento : 0.8056
Precisión en set de test : 0.75
Degradación:  6.9 %


## 5.5 Random Forest
Encuentra el mejor modelo de rándom forest posible

In [ ]:
modelo = ensemble.RandomForestClassifier(n_estimators = 500,
                                         max_features = "auto",
                                         max_depth = 3,
                                         min_samples_split = 20,
                                         min_samples_leaf = 5,
                                         min_impurity_decrease = 0.005)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisión en set de entrenamiento : 0.7917
Precisión en set de test : 0.7552
Degradación:  4.61 %


## 5.6 Gradient Boosting
Encuentra el mejor modelo de gradient boosting posible.

In [ ]:
modelo = ensemble.GradientBoostingClassifier(n_estimators = 250,
                                             learning_rate = 0.005,
                                             max_depth = 3,
                                             min_samples_split = 20,
                                             min_samples_leaf = 5,
                                             min_impurity_decrease = 0.0005,
                                             random_state = 0)

# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

Precisión en set de entrenamiento : 0.8073
Precisión en set de test : 0.7812
Degradación:  3.23 %
